# Experiment 5.0 - Inception Resnet v2 - extract Features

Reproduce Results of [Transfer learning with deep convolutional neural network for liver steatosis assessment in ultrasound images](https://pubmed.ncbi.nlm.nih.gov/30094778/). We used a pre-trained CNN to extract features based on B-mode images. 

The CNNfeatures are extracted using the pretrained Inception-Resnet-v2 implemented in Keras.
See reference: https://jkjung-avt.github.io/keras-inceptionresnetv2/

![Screen Shot 2020-10-12 at 2 57 22 PM](https://user-images.githubusercontent.com/23482039/95781182-47437700-0c9b-11eb-8826-594811ba3322.png)


In [1]:
import sys
import random
sys.path.append('../src')

import warnings
warnings.filterwarnings("ignore") 

import pickle
import pandas as pd

import numpy as np
import mlflow
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tqdm import tqdm

from utils.compute_metrics import get_metrics, get_majority_vote,log_test_metrics
from utils.dataframe_creation import create_dataframe_preproccessing
from sklearn.preprocessing import StandardScaler
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
from pprint import pprint
from itertools import product
from keras.preprocessing.image import ImageDataGenerator


## 1. Feature Extraction

In [2]:
M, N= 434, 636 # ultrasound image dimension
#In the paper, the authors extract the features directly from the pretrained inception resnet_v2
# pooling: Optional pooling mode for feature extraction when include_top is False.
#'max' means that global max pooling will be applied.
net = InceptionResNetV2(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(M,N,3),
                        pooling= 'avg')
dataset = create_dataframe_preproccessing()
dataset.head()

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,labels,fat,fname
0,1,0,3,../data/02_interim/raw_images/1/P1_image1.jpg
1,1,0,3,../data/02_interim/raw_images/1/P1_image2.jpg
2,1,0,3,../data/02_interim/raw_images/1/P1_image3.jpg
3,1,0,3,../data/02_interim/raw_images/1/P1_image4.jpg
4,1,0,3,../data/02_interim/raw_images/1/P1_image5.jpg


In [3]:
dataset = dataset.astype({"labels": str})
datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_dataframe(dataset, x_col='fname',y_col='labels',target_size=(M,N),shuffle=False, batch_size =25)

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Found 550 validated image filenames belonging to 2 classes.


In [4]:
max_pool_dim = 1536
features = np.zeros(shape=(len(dataset), max_pool_dim))
batch_size = 25
assert len(dataset)%batch_size ==0, '550 should be divisable by batch_SIZE'

for i, (inputs_batch, labels_batch) in enumerate(tqdm(generator)):
    features[i*batch_size:(i+1)*batch_size] = net(inputs_batch)
    if (i+1)*batch_size == 550:
        break

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 95%|█████████▌| 21/22 [08:57<00:25, 25.58s/it]


## 2. Save features

In [5]:
df_features = pd.DataFrame(features)
df_features= pd.concat([dataset['id'], dataset['labels'], df_features], axis=1)
df_features[["labels"]] = df_features[["labels"]].apply(pd.to_numeric)
df_features.head()

,id,labels,0,1,2,3,4,5,6,7,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,1,0,0.101244,0.197340,0.122139,0.026230,0.091077,0.200453,0.342147,0.110519,...,0.022503,0.465169,0.219377,0.280133,0.218263,0.326974,0.030053,0.053427,0.127963,0.424772
1,1,0,0.127029,0.186655,0.144265,0.022980,0.083105,0.163568,0.326425,0.149001,...,0.008748,0.422535,0.198117,0.263455,0.224313,0.324824,0.042945,0.050160,0.109165,0.411583
2,1,0,0.120061,0.218817,0.139202,0.033013,0.130737,0.161452,0.298092,0.177543,...,0.009677,0.474711,0.248197,0.281001,0.218283,0.369392,0.065059,0.073969,0.122241,0.440500
3,1,0,0.133891,0.166143,0.131115,0.026295,0.138991,0.165849,0.262839,0.129531,...,0.007220,0.447484,0.233246,0.241784,0.186390,0.368073,0.051665,0.062716,0.108734,0.448344
4,1,0,0.098471,0.177389,0.125236,0.013000,0.124575,0.181096,0.266885,0.171505,...,0.013480,0.474544,0.237659,0.317590,0.189135,0.290581,0.041239,0.052399,0.097868,0.454580


In [6]:
# Create a dict for the scattering features, and parameters
inception_dict_tensor = {
                'features':df_features,
                'label' : dataset['labels'],
                'pid' : dataset['id'],}

with open('../data/03_features/inception_dict_tensor_avg_pooling.pickle', 'wb') as handle:
    pickle.dump(inception_dict_tensor, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
